# Exploration of actors repartition based upon gender

In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [14]:
# Data source: http://www.cs.cmu.edu/~ark/personas/

# Define the data path
CLEAN_DATA_PATH = 'clean_data/'

In [50]:
# Function: filter NaN of a given column
def filter_NaN(dataframe, column_to_filter):
    '''
    Create a new dataframe filtering out the NaN values
    of the given column of the given dataframe 
    :param dataframe: pandas.DataFrame
    :param column_to_filter: str
    :return: new dataframe
    '''
    new_df = dataframe.dropna(axis=0, subset=column_to_filter)
    print(f"Cleaning of {column_to_filter} : {new_df.shape[0]} entries left")
    return new_df    

In [55]:
# Load the data
characters_df = pd.read_csv(CLEAN_DATA_PATH + "characters.csv")
display(characters_df)

initial_entries_number = characters_df.shape[0]
print(f"Initial entries: {initial_entries_number}")

# Remove NaN from column actor_gender
filt_characters_df = filter_NaN(characters_df, "actor_gender")

# Remove NaN from actor_age
filt_characters_df = filter_NaN(filt_characters_df, "actor_age")

# Remove NaN from movie_release_date
filt_characters_df = filter_NaN(filt_characters_df, "movie_release_date")

,movie_wiki_id,movie_id,movie_release_date,name,actor_birth_date,actor_gender,actor_height,actor_ethinicity_id,actor_name,actor_age,actor_map_id,id,actor_id
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
450664,913762,/m/03pcrp,1992-05-21,Elensh,1970-05,F,NaN,NaN,Dorothy Elias-Fahn,NaN,/m/0kr406c,/m/0kr406h,/m/0b_vcv
450665,913762,/m/03pcrp,1992-05-21,Hibiki,1965-04-12,M,NaN,NaN,Jonathan Fahn,27.0,/m/0kr405_,/m/0kr4090,/m/0bx7_j
450666,28308153,/m/0cp05t9,1957,NaN,1941-11-18,M,1.730,/m/02w7gg,David Hemmings,15.0,/m/0g8ngmc,NaN,/m/022g44
450667,28308153,/m/0cp05t9,1957,NaN,NaN,NaN,NaN,NaN,Roberta Paterson,NaN,/m/0g8ngmj,NaN,/m/0g8ngmm


Initial entries: 450669
Cleaning of movie_release_date : 440674 entries left
Cleaning of actor_age : 292556 entries left
Cleaning of actor_gender : 290770 entries left


Note that the order of column filtering of NaN is independent of the number of entries we obtain at the end.

#### Comparison of actresses and actors number over the years 

In [ ]:
# def get_year(dataframe, x):
#     try:
#         return pd.to_datetime(dataframe["movie_release_date"]).year()
#     except return x

# ## try except 

# # Uniformize movie_release_date to string containing only the year
# characters_df["movie_release_date"] = characters_df["movie_release_year"].apply(lambda x: get_year(x))
# characters_df["movie_release_year"] = 